In [26]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import scipy.optimize as opt
from scipy.integrate import odeint, quad
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit

from pathlib import Path
import os
import sys
from byutpl.properties import water as water

In [27]:
# ----- Constants ----- #
# heat exchanger physical parameters
di = .206 * 2.54 / 100                  # m
do = .25 * 2.54 / 100                   # m
L = 14 * 2.54 / 100                     # m
k = 13.4                                # W/m.K 316 SS
g = 9.81                                # m/s^2
N = 56                                  # number of tubes    

# calculate the heat transfer area
Ai = .25 * np.pi * di**2
Ao = .25 * np.pi * do**2

# ----- Functions ----- #
def hi(Qi,Ti):
    # calculate the velocity
    v = Qi / Ai

    # calculate the Reynolds number
    Re = water.ldn(Ti) * v * di / water.lvs(Ti)

    # calculate the Nusselt number
    if Re < 10000:
        Nu = 3.66
    else:
        Nu = .023 * (Re**.8) * water.lpr(Ti)**.4

    # calculate heat transfer coefficient
    h = Nu * water.ltc(Ti) / di
    return h

def ho(Ps,Ts):
    # pull in the values
    rhol = water.ldn(Ts)
    rhov = water.vdn(Ts,Ps)
    kl = water.ltc(Ts)
    mul = water.lvs(Ts)
    Tsat = water.tsat(Ps)
    cpl = water.lcp(Ts) / water.mw
    hfg = water.hvp(Ts) / water.mw

    # find Ja
    Ja = cpl * (Tsat - Ts) / hfg

    # calculate the condensation energy
    hfp = hfg * (1 + (.68 * Ja))

    # calculate the heat transfer coefficient
    h = .729 * (rhol * g * (rhol - rhov) * hfp * kl**3 / (N * mul * (Tsat - Ts) * do))**.25
    return h

hi_vec = np.vectorize(hi)
ho_vec = np.vectorize(ho)


def model(inputs,Rf):
    Qwd,Psd,Tweffd = inputs

    #                       |                 |                                 |
    #      convection_inner | fouling_inner   |             conduction          | convection_outer
    #                       |                 |                                 |
    UA = ((hi_vec(Qwd, Tweffd) * Ai)**-1 + (Rf / Ai) + (np.log(do / di) / (2 * np.pi * k * L)) + (ho_vec(Psd, Tweffd) / Ao)**-1)**-1
    # print(UA)
    return UA

# def model(Qw,Ps,Tweff,Rf):

#     #                       |                 |                                 |
#     #      convection_inner | fouling_inner   |             conduction          | convection_outer
#     #                       |                 |                                 |
#     UA = ((hi(Qw, Tweff) * Ai)**-1 + (Rf / Ai) + (np.log(do / di) / (2 * np.pi * k * L)) + (ho(Ps, Tweff) / Ao)**-1)**-1
#     return UA

# model = np.vectorize(model)

# correlation for if we want to try to fit both fouling factors

# Qi,Qo = Qs
# #                       |                 |                                  |                 |
# #      convection_inner | fouling_inner   |             conduction           |  fouling_outer  | convection_outer
# #                       |                 |                                  |                 |
# sumR = (hi(Qi) * Ai)**-1 + (Rfi / Ai) + (np.log(do / di) / (2 * np.pi * k * L)) + (Rfo / Ao) + (ho(Qo) / Ao)**-1

$$UA = (\Sigma R)^{-1} = \frac{1}{h_i(\dot V)A_i} + \frac{R_{f,i}^"}{A_i} + \frac{ln(d_o / d_i)}{2\pi kL}+ \frac{R_{f,o}^"}{A_o} + \frac{1}{h_o(\dot V)A_o}$$

In [28]:
data1 = pd.read_csv('data/Trial1.csv')
data2 = pd.read_csv('data/Trial2.csv')
data3 = pd.read_csv('data/Trial3.csv')
data4 = pd.read_csv('data/Trial4.csv')
data5 = pd.read_csv('data/Trial5.csv')
data6 = pd.read_csv('data/Trial6.csv')
data7 = pd.read_csv('data/Trial7.csv')

data_collection = np.array([data1,data2,data3,data4,data5,data6,data7])

print(data1.keys())

Index(['Time (sec)', 'Water Level (ft)', 'Water Flowrate (GPM)',
       'House Steam Pressure (psig)', 'Steam Pressure (psig)',
       'Inlet Water Temperature (C)', 'Outlet Water Temperature (C)',
       'Makeup Temperature (C)', 'Makeup Flowrate (L/min)',
       'Ambient Temperature (C)', 'Ambient Pressure (kPa)',
       'Flow Setpoint (GPM)', 'Flow Control Output (%)', 'Level Setpoint (ft)',
       'Level Control Output (%)', 'Steam Setpoint (psig)',
       'Steam Control Output (%)', 'Tube-Side Pressure Drop (psig)'],
      dtype='object')


In [ ]:
qs = np.array([])
Twout = np.array([])
Twin = np.array([])
Ps = np.array([])

for i, df in enumerate(data_collection):
    qs = np.append(qs,df[:,2])
    Twout = np.append(Twout,df[:,6])
    Twin = np.append(Twin,df[:,5])
    Ps = np.append(Ps,df[:,4])

    
Tavg = (Twout + Twin) / 2

# conver the data to SI units
qs_good = qs * 6.30901964e-5                # gal/min to m^3/s
Ps_good = (Ps + 14.7) * 101325 / 14.7       # psig to Pa
Cpw = water.lcp(Tavg + 273.15) / water.mw   # J/kg.K

tsat = np.vectorize(water.tsat)

Tsat = tsat(Ps_good)

# calculate the delta T values
dT1 = Tsat - Twout
dT2 = Tsat - Twin

# calcualate the delta T log mean
dTlm = (dT1 - dT2) / np.log(dT1 / dT2)

# calculate the mass flow rate of the water 
rho = water.ldn(Tavg + 273.15)
m = qs_good / rho

# find the heat transfer
Q = -m * Cpw * (Twin - Twout)

# calculate the heat transfer coefficient
UA = Q / dTlm

In [ ]:
# # fit the data with the model
# Rf = curve_fit(model, (qs_good,Ps_good,Tavg + 273.15), UA)

xdata = np.array([qs_good, Ps_good, Tavg + 273.15])  # Stack inputs correctly

# print(xdata)
Rf, _ = curve_fit(model, xdata, UA)
Rf = Rf[0]

print(Rf)

0.1413151788747382
